In [16]:
#dependencies
import pandas as pd
import math


In [3]:
#Bring in the dataset
gss = pd.io.stata.read_stata('GSS1996.dta', convert_categoricals=False)
gss.to_csv('gss_converted.csv')

In [33]:
#drop all instances where angry event was not recalled
gss=gss.dropna(subset=["chnang7"])